# Problems with 1-hot encoding

## It creates models that don't fit well

When dealing with categorical features the literature on machine learning teaches us to apply 1-hot encoding without much discussion of validity of this approach. It was also an article of faith for me until my fellow data scientist challenged my beliefs. Instead of using 1-hot encoding for my regression model, he suggested that I fit several models, one for each category value. Indeed, if I'm using 1-hot encoding, this is equivalent to adding a category-dependent value to the logit:

$$y = \theta_1 x + \theta_0 + \theta_c $$

The hypothesis in this case is that only the intercept depends on the category value, and the slope does not depend on the category. And while this may end up being true, the more general model is when both the intercept and the slope depend on the category:

$$y = \theta_1(c) x + \theta_0(c) $$

## The distribution of categorical variables is different from the real-valued numerical variables

First, the simple fact, that multiple categorical variables can be reduced to one categorical variable that has its values as all possible combinations of the multiple categories. Thus, is we have two categorical variables with 2 and 3 values respectively, it can be reduced to having one categorical variable with 6 possible values (regardless whether they all actually occur in the data set or not)

The issue is that both likelihood and the posterior distribution is different for real-valued numerical features and the categorical features. For the real-valued numerical features the likelihood function is Normal distribution, for the categorical feature it is Categorical (or Multinomial) distribution. The posterior is Normal for $\mu$ and Inverse-Gamma for $\sigma^2$ for numerical features, but it is Dirichlet distribution for the categorical features. 

For this reason it is easier to represent the joined likelihood $p(x, c | \theta)$ in the conditional form $p(x | c, \theta) p(c|\theta)$ or $p(c | x, \theta) p(x|\theta)$. Here x is a vector of real-valued numerical features, c is a combined categorical feature, and $\theta$ is the vector of model parameters. If we are working on a supervised learning problem, then the target is included together with x in case of regression and c in case of classification.





## First approach

In this approach we represent the likelihood as $p(x | \theta_x(c)) p(c|\theta_c)$. Here we split the parameter vector $\theta$ to two: $\theta_c$ are the parameters of the Categorical distribution, and $\theta_x$ are the parameters of the multivariate Normal distribution. In this model we separately fit the categorical model and multiple Normal distributions. Note, that this approach is equivalent to Gaussian mixture model and soft K-means clustering.

If we are working on a regression problem, this will be equivalent to fitting multiple regression model for each category combination, but weigh them based on the probability of the given category combinations. In case of classification model, each fitted Normal distribution defines the probability of belonging to a cluster, correlated to the category value, then Bayesian classifier can be used to fit the model. 

The main weakness with this approach is that we cannot make any prediction for the category values c for which we have no data. But this can be easily solved by regularizing the Multinomial distribution by using a prior Dirichlet distribution with parameters $\alpha > 1$ to make sure the posterior distribution does not encourage zero probability of any category value, even in case where we have no data for that value.

Another weakness is that this model may be overfitting if there are too few examples for a category value. Regularization will help up to a certain degree, but a better way would be to use the second approach.

## Second approach
Here the likelihood can be represented as $p(c|x)p(x|\theta_x)$. Note, that here the parameters of the categorical distribution depend on x. Another words, the probability of a given category value is defined by where in the x-space are we. In this case you can use local or even non-parametric methods to estimate p(c|x). For example, we can use KNN algorithm to predict the category based on the values of the categorical features of K nearest points in K-space. This algorithm, however, will not work if the points of different categories are close together. For example, a male with an income of \$49,000 and a female with an income of \$49,500.

## Conclusion
In this article I pointed out some of the weaknesses of 1-hot encoding and the probabilistic approaches that can help us overcome it. Next steps would be to develop efficient algorithms that would help us use these approaches in practice.